
![SciUnit Logo](https://raw.githubusercontent.com/scidash/assets/master/logos/sciunit.png)

<a href="https://colab.research.google.com/github/ChihweiLHBird/sciunit/blob/dev/docs/QuickTutorialsNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 1. What is SciUnit?

### SciUnit is a framework for validating scientific models by creating experimental-data-driven unit tests.  

### Everyone hopes that their model has some correspondence with reality.  Usually, checking whether this is true is done informally.
### SciUnit makes this formal and transparent.  

The figure below illustrates both the results you get (center table) and the relationship between the components listed here. 

![Cosmo Example](https://raw.githubusercontent.com/scidash/assets/master/figures/cosmo_example.png)

# Chapter 2. Writing a `model` and `test` in SciUnit from scratch

In [25]:
# uncomment the follwing line if ruuning this notebook on Google Colab
!pip install -q sciunit

import sciunit

### SciUnit works by making models declare and implement capabilities that tests use to interact with those models.  
Each `capability` is a subclass of `sciunit.Capability`, and contains one or more unimplemented methods.  Here we define a simple capability through which a model can return a single number.  

In [26]:
class ProducesNumber(sciunit.Capability):
    """An example capability for producing some generic number."""

    def produce_number(self):
        """The implementation of this method should return a number."""
        raise NotImplementedError("Must implement produce_number.")

### SciUnit models subclass `sciunit.Model` as well as each `sciunit.Capability` they aim to implement. 
Here we create a trivial model class that is instantiated with a single constant.  

In [27]:
from sciunit.capabilities import ProducesNumber # One of many potential model capabilities.

In [28]:
class ConstModel(sciunit.Model, 
                 ProducesNumber):
    """A model that always produces a constant number as output."""
    
    def __init__(self, constant, name=None):
        self.constant = constant 
        super(ConstModel, self).__init__(name=name, constant=constant)

    def produce_number(self):
        return self.constant

### A `model` we want to test is always an instance (with specific model arguments) of a more generic `model` class.  
Here we create an instance of `ConstModel` that will always produce the number 37 and give it a name.  

In [29]:
const_model_37 = ConstModel(37, name="Constant Model 37")

### A SciUnit test class must contain:
1. the capabilities a model requires to take the test.  
2. the type of score that it will return
3. an implementation of `generate_prediction`, which will use the model's capabilities to get some values out of the model.
4. an implementaiton of `compute_score`, to use the provided observation and the generated prediction to compute a sciunit `Score`.

In [30]:
from sciunit.scores import BooleanScore # One of several SciUnit score types.  

In [31]:
class EqualsTest(sciunit.Test):
    """Tests if the model predicts 
    the same number as the observation."""   
    
    required_capabilities = (ProducesNumber,) # The one capability required for a model to take this test.  
    score_type = BooleanScore # This test's 'judge' method will return a BooleanScore.  
    
    def generate_prediction(self, model):
        return model.produce_number() # The model has this method if it inherits from the 'ProducesNumber' capability.
    
    def compute_score(self, observation, prediction):
        score = self.score_type(observation['value'] == prediction) # Returns a BooleanScore. 
        score.description = 'Passing score if the prediction equals the observation'
        return score

### A SciUnit test is a specific instance of a `test` class, parameterized by the observation (i.e. the empirical data that the `model` aims to recapitulate).  
Here we create a test instance parameterized by the observation 37.0.  

In [32]:
equals_37_test = EqualsTest({'value':37}, name='=37')

### Every test has a `judge` method which executes the test and returns a `score` for the provide model.  
Here we judge the model we just created using the test we just created.  The `judge` method does a lot of things behind the scenes:  
1. It checks to makes sure that your `model` expresses each `capability` required to take the test. It doesn't check to see if they are implemented correctly (how could it know?) but it does check to make sure the `model` at least claims (through inheritance) to express each `capability`. The required capabilities are none other than those in the test's `required_capabilities` attribute. Since `ProducesNumber` is the only required capability, and the `ConstModel` class inherits from the corresponding capability class, that check passes.
2. It calls the test's `generate_prediction` method, which uses the model's capabilities to make the model return some quantity of interest, in this case a characteristic number.
3. It calls the test's `compute_score` method, which compares the observation the test was instantiated with against the prediction returned in the previous step. This comparison of quantities is cast into a score (in this case, a `BooleanScore`), bound to some `model` output of interest (in this case, the number produces by the `model`), and that `score` object is returned.
4. The `score` returned is checked to make sure it is of the type promised in the class definition, i.e. that a `BooleanScore` is returned if a `BooleanScore` is listed in the `score_type` attribute of the `test`.
5. The `score` is bound to the `test` that returned it, the `model` that took the `test`, and the prediction and observation that were used to compute it.

In [33]:
score = equals_37_test.judge(const_model_37)

### A score is an object containing information about the result of the test, and the provenance of that result.  
Printing the `score` just prints a representation of its value (for a `BooleanScore`, `True` has the representation 'Pass')

In [34]:
score

Pass

We can also summarize the `score` in its entirety, printing information about the associated `model` and `test`.  

In [35]:
score.summarize()

How was that score computed again?  

In [36]:
score.describe()

### Several logically related tests can be grouped using a `TestSuite`.  
These can be instances of the same test class (instantiated with different observations) or instances of different test classes.  Anything tests that you think belongs together can be part of a TestSuite.  A test can be a part of many different suites at once.  

In [37]:
equals_1_test = EqualsTest({'value':1}, name='=1') # Test that model output equals 1.  
equals_2_test = EqualsTest({'value':2}, name='=2') # Test that model output equals 2.  

equals_suite = sciunit.TestSuite([equals_1_test, equals_2_test, equals_37_test], name="Equals test suite")

Now we can test our model using this TestSuite, and display the results.  

In [38]:
score_matrix = equals_suite.judge(const_model_37)
score_matrix

,=1,=2,=37
Constant Model 37,Fail,Fail,Pass


We can create more models and subject those to the test suite to get a more extensive score matrix.

In [39]:
const_model_1 = ConstModel(1, name='Constant Model 1')
const_model_2 = ConstModel(2, name='Constant Model 2')
score_matrix = equals_suite.judge([const_model_1, const_model_2, const_model_37])
score_matrix

,=1,=2,=37
Constant Model 1,Pass,Fail,Fail
Constant Model 2,Fail,Pass,Fail
Constant Model 37,Fail,Fail,Pass


We can also examine the results only for one of the tests in the suite.

In [40]:
score_matrix[equals_1_test]

Constant Model 1     Pass
Constant Model 2     Fail
Constant Model 37    Fail
Name: =1, dtype: object

Or examine the results only for one of the models.  

In [41]:
score_matrix[const_model_2]

=1     Fail
=2     Pass
=37    Fail
Name: Constant Model 2, dtype: object

# Chapter 3. Testing with help from the SciUnit standard library

In [ ]:
import sciunit

### In this chapter we will use the same toy model in Chapter 1 but write a more interesting test with additional features included in SciUnit. 

In [ ]:
from sciunit.models.examples import ConstModel # One of many dummy models included for illustration.  
const_model_37 = ConstModel(37, name="Constant Model 37")

Now let's write a test that validates the `observation` and returns more informative `score` type.

In [ ]:
from sciunit.capabilities import ProducesNumber
from sciunit.scores import ZScore # One of many SciUnit score types.  
from sciunit.errors import ObservationError # An exception class raised when a test is instantiated 
                                     # with an invalid observation.
    
class MeanTest(sciunit.Test):
    """Tests if the model predicts 
    the same number as the observation."""   
    
    required_capabilities = (ProducesNumber,) # The one capability required for a model to take this test.  
    score_type = ZScore # This test's 'judge' method will return a BooleanScore.  
    
    def validate_observation(self, observation):
        if type(observation) is not dict:
            raise ObservationError("Observation must be a python dictionary")
        if 'mean' not in observation:
            raise ObservationError("Observation must contain a 'mean' entry")
        
    def generate_prediction(self, model):
        return model.produce_number() # The model has this method if it inherits from the 'ProducesNumber' capability.
    
    def compute_score(self, observation, prediction):
        score = ZScore.compute(observation,prediction) # Compute and return a ZScore object.  
        score.description = ("A z-score corresponding to the normalized location of the observation "
                             "relative to the predicted distribution.")
        return score

We've done two new things here:
- The optional `validate_observation` method checks the `observation` to make sure that it is the right type, that it has the right attributes, etc.  This can be used to ensures that the `observation` is exactly as the other core test methods expect.  If we don't provide the right kind of observation:
```python
-> mean_37_test = MeanTest(37, name='=37')
ObservationError: Observation must be a python dictionary
```
then we get an error.  In contrast, this is what our test was looking for:

In [ ]:
observation = {'mean':37.8, 'std':2.1}
mean_37_test = MeanTest(observation, name='=37')

- Instead of returning a `BooleanScore`, encoding a `True`/`False` value, we return a `ZScore` encoding a more quantitative summary of the relationship between the observation and the prediction.  When we execute the test:

In [ ]:
score = mean_37_test.judge(const_model_37)

In [ ]:
score.summarize()

In [ ]:
score.describe()

# Chapter 4 Example of RunnableModel and Backend

Beside the usual model in previous sections, let’s create a model that run a Backend instance to simulate and obtain results.

Firstly, import necessary components from SciUnit package.

In [42]:
import sciunit, random
from sciunit import Test
from sciunit.capabilities import Runnable
from sciunit.scores import BooleanScore
from sciunit.models import RunnableModel
from sciunit.models.backends import register_backends, Backend

Let’s define subclasses of SciUnit Backend, Test, and Model.

Note:
1. A SciUnit Backend subclass should implement ``_backend_run`` method. 
2. A SciUnit Backend subclass should implement ``run`` method.

In [43]:
class RandomNumBackend(Backend):
    '''generate a random integer between min and max'''

    def set_run_params(self, **run_params):

        # get min from run_params, if not exist, then 0.
        self.min = run_params.get('min', 0)

        # get max from run_params, if not exist, then self.min + 100.
        self.max = run_params.get('max', self.min + 100)

    def _backend_run(self):
        # generate and return random integer between min and max.
        return random.randint(self.min, self.max)

class RandomNumModel(RunnableModel):
    """A model that always produces a constant number as output."""

    def run(self):
        self.results = self._backend.backend_run()


class RangeTest(Test):
    """Tests if the model predicts the same number as the observation."""

    # Default Runnable Capability for RunnableModel
    required_capabilities = (Runnable,)

    # This test's 'judge' method will return a BooleanScore.
    score_type = BooleanScore

    def generate_prediction(self, model):
        model.run()
        return model.results

    def compute_score(self, observation, prediction):
        score = BooleanScore(
            observation['min'] <= prediction and observation['max'] >= prediction
        )
        return score

Let’s define the model instance named ``model 1.``

In [44]:
model = RandomNumModel("model 1")

We must register any backend isntance in order to use it in model instances.

``set_backend`` and ``set_run_params`` methods can help us to set the run-parameters in the model and its backend.

In [45]:
register_backends({"Random Number": RandomNumBackend})
model.set_backend("Random Number")
model.set_run_params(min=1, max=10)

Next, create an observation that requires the generated random integer between 1 and 10 and a test instance that use the observation and against the model

In [46]:
observation = {'min': 1, 'max': 10}
oneToTenTest = RangeTest(observation, "test 1")
score = oneToTenTest.judge(model)

print the score, and we can see the result.

In [47]:
print(score)

Pass
